In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader, Subset
import tqdm
import json
import datasets
from typing import List
import os
import pandas as pd
import tiktoken
import inspect
import math

In [3]:
!mkdir data/

mkdir: cannot create directory ‘data/’: File exists


In [4]:
encoding = tiktoken.get_encoding("gpt2")

In [5]:
encoding.n_vocab

50257

In [6]:
encoding._special_tokens

{'<|endoftext|>': 50256}

In [7]:
class Tokenizer:
    def __init__(self, tokenizer_model="gpt2"):
        gpt2_enc = tiktoken.get_encoding(tokenizer_model)
        self.enc = tiktoken.Encoding(
            name=tokenizer_model,
            pat_str=gpt2_enc._pat_str,
            mergeable_ranks=gpt2_enc._mergeable_ranks,
			special_tokens={
                **gpt2_enc._special_tokens,
                "PAD": 50257,
			},
		)
        self.tokenizer_model = tokenizer_model

        self.n_words = self.enc.n_vocab
        self.bos_id = None
        self.eos_id = self.enc.eot_token
        self.pad_id = self.enc._special_tokens["PAD"]

    def encode(self, s: str, bos: bool = False, eos: bool = False) -> List[int]:
        t = self.enc.encode(s)
        if bos and self.bos_id is not None:
            t = [self.bos_id] + t
        if eos and self.eos_id is not None:
            t = t + [self.eos_id]
        return t

    def decode(self, tokens: List[int]) -> str:
        return self.enc.decode(tokens)

In [8]:
tokenizer = Tokenizer(tokenizer_model="gpt2")

In [9]:
tokenizer.n_words

50258

In [10]:
vocab_size = 50304
batch_size = 16
block_size = 512
max_iters = 1
eval_interval = 1000
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 256
n_embd = 512
n_head = 8
n_layer = 8
gqa_kv_head = 4
dropout = 0.3

target_batch_size = 4096
gradient_accumulation_steps = target_batch_size // batch_size
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95

In [11]:
gradient_accumulation_steps

256

In [12]:
torch.set_float32_matmul_precision('high')

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if torch.cuda.device_count() > 1:
#     print(f"Using {torch.cuda.device_count()} GPUs!")

In [14]:
def encode(s): return tokenizer.encode(s, bos=False, eos=False)

def decode(l):
	try:
		return tokenizer.decode(l)
	except:
		return ""

In [15]:
ds = datasets.load_dataset("roneneldan/TinyStories")

In [16]:
ds = ds.with_format("torch")

In [17]:
ds['train'][1]

{'text': 'Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.\n\nOne day, Beep was driving in the park when he saw a big tree. The tree had many leaves that were falling. Beep liked how the leaves fall and wanted to play with them. Beep drove under the tree and watched the leaves fall on him. He laughed and beeped his horn.\n\nBeep played with the falling leaves all day. When it was time to go home, Beep knew he needed more fuel. He went to the fuel place and got more healthy fuel. Now, Beep was ready to go fast and play again the next day. And Beep lived happily ever after.'}

In [18]:
def collate_fn(batch):
    texts = [encode(item['text'])[:block_size+1] for item in batch]  # Get one extra token for targets
    # Create inputs and targets with a 1-token shift
    batch_data = []
    for text in texts:
        if len(text) <= 1:  # Skip sequences that are too short
            continue
        
        # Use all but the last token as input
        input_text = text[:-1]
        # Use all but the first token as target (shifted by 1)
        target_text = text[1:]
        
        # Pad to block_size if needed
        if len(input_text) < block_size:
            input_text = input_text + [0] * (block_size - len(input_text))
        if len(target_text) < block_size:
            target_text = target_text + [0] * (block_size - len(target_text))
            
        batch_data.append({
            'input': torch.tensor(input_text, dtype=torch.long),
            'target': torch.tensor(target_text, dtype=torch.long)
        })
    
    # Stack all batch items
    if not batch_data:  # Handle empty batch case
        return None
    
    return {
        'input': torch.stack([item['input'] for item in batch_data]),
        'target': torch.stack([item['target'] for item in batch_data])
    }

In [19]:
eval_iters

256

In [20]:
subset_indices = list(range(eval_iters))
dataset_valid = Subset(ds['validation'], subset_indices)

In [21]:
train_dataloader = DataLoader(ds['train'], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_dataloader = DataLoader(dataset_valid, batch_size=batch_size, collate_fn=collate_fn)

In [22]:
class RMSNorm(nn.Module):
    """
    Root Mean Square Layer Normalization as described in the GPT-4 paper.
    """
    def __init__(self, dim, eps=1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def forward(self, x):
        # Calculate RMS
        rms = torch.sqrt(torch.mean(x ** 2, dim=-1, keepdim=True) + self.eps)
        # Normalize and scale
        x_norm = x / rms
        # Apply learnable parameters
        return self.weight * x_norm

def precompute_freqs_cis(dim, seq_len, theta=10000.0):
    """
    Precompute the frequency tensor for complex rotation for rotary embeddings.
    """
    # Compute frequencies
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2).float() / dim))
    # Create position sequence
    t = torch.arange(seq_len, dtype=torch.float)
    # Outer product of position and frequencies
    freqs = torch.outer(t, freqs)
    # Create complex numbers as pairs of (cos, sin)
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs)
    return freqs_cis

def apply_rotary_emb(x, freqs_cis):
    """
    Apply rotary embeddings to input tensors using the given frequencies.
    """
    # Reshape for broadcasting, assuming x has shape (..., seq_len, dim)
    # and freqs_cis has shape (seq_len, dim/2)
    seq_len = x.shape[-2]
    x_complex = torch.view_as_complex(x.float().reshape(*x.shape[:-1], -1, 2))
    # Complex multiply
    freqs_cis = freqs_cis[:seq_len]
    x_rotated = x_complex * freqs_cis.unsqueeze(0)
    # Convert back to real
    x_out = torch.view_as_real(x_rotated).flatten(-2)
    return x_out.type_as(x)

class GroupedQueryAttention(nn.Module):
    """
    Grouped Query Attention as described in GQA papers.
    Similar to MHA but with fewer key/value heads than query heads.
    """
    def __init__(self, n_embd, n_head, n_query_groups, dropout=0.1):
        super().__init__()
        # Ensure n_head is divisible by n_query_groups
        assert n_head % n_query_groups == 0, "n_head must be divisible by n_query_groups"
        
        self.n_embd = n_embd
        self.n_head = n_head
        self.n_query_groups = n_query_groups
        self.head_dim = n_embd // n_head
        
        # Query projection for all heads
        self.q_proj = nn.Linear(n_embd, n_embd, bias=False)
        
        # Shared key/value projections (fewer heads than queries)
        kv_dim = n_embd * n_query_groups // n_head
        self.k_proj = nn.Linear(n_embd, kv_dim, bias=False)
        self.v_proj = nn.Linear(n_embd, kv_dim, bias=False)
        
        self.out_proj = nn.Linear(n_embd, n_embd, bias=False)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, freqs_cis, mask=None):
        B, T, C = x.shape
        H = self.n_head
        KV_H = self.n_query_groups
        head_dim = C // H
        
        # Project queries, keys, values
        q = self.q_proj(x).view(B, T, H, head_dim)  # (B, T, H, head_dim)
        k = self.k_proj(x).view(B, T, KV_H, head_dim)  # (B, T, KV_H, head_dim)
        v = self.v_proj(x).view(B, T, KV_H, head_dim)  # (B, T, KV_H, head_dim)
        
        # Apply rotary embeddings
        q_rot = apply_rotary_emb(q, freqs_cis)
        k_rot = apply_rotary_emb(k, freqs_cis)
        
        # Repeat k, v for heads that share the same key/value heads
        heads_per_group = H // KV_H
        # Reshape for broadcasting: (B, T, KV_H, 1, head_dim) -> (B, T, KV_H, heads_per_group, head_dim)
        k_expanded = k_rot.unsqueeze(3).expand(B, T, KV_H, heads_per_group, head_dim)
        v_expanded = v.unsqueeze(3).expand(B, T, KV_H, heads_per_group, head_dim)
        
        # Reshape to match expected shape for attention
        k_final = k_expanded.reshape(B, T, H, head_dim)
        v_final = v_expanded.reshape(B, T, H, head_dim)
        
        # Transpose to get dimensions [B, H, T, head_dim]
        q = q_rot.transpose(1, 2)  # (B, H, T, head_dim)
        k = k_final.transpose(1, 2)  # (B, H, T, head_dim)
        v = v_final.transpose(1, 2)  # (B, H, T, head_dim)
        
        # Compute attention scores
        scale = head_dim ** -0.5
        attn = (q @ k.transpose(-2, -1)) * scale  # (B, H, T, T)
        
        # Apply causal mask
        if mask is not None:
            attn = attn + mask.unsqueeze(0).unsqueeze(0)  # (B, H, T, T)
        
        # Compute attention weights and apply to values
        attn = F.softmax(attn, dim=-1)
        attn = self.dropout(attn)
        y = attn @ v  # (B, H, T, head_dim)
        
        # Reshape to original dimensions
        y = y.transpose(1, 2).contiguous().view(B, T, C)  # (B, T, C)
        
        # Final projection
        y = self.out_proj(y)
        return y

class Block(nn.Module):
    """Transformer block with RMSNorm, RoPE, and GQA"""
    def __init__(self, n_embd, n_head, n_query_groups):
        super().__init__()
        # Grouped Query Attention
        self.attn = GroupedQueryAttention(
            n_embd=n_embd,
            n_head=n_head,
            n_query_groups=n_query_groups,
            dropout=dropout
        )
        
        # Feed-forward network
        self.ffwd = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),  # Changed from ReLU to GELU
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
        
        # RMSNorm layers instead of LayerNorm
        self.ln1 = RMSNorm(n_embd)
        self.ln2 = RMSNorm(n_embd)
        
    def forward(self, x, freqs_cis, causal_mask):
        # Pre-RMSNorm for attention
        x_ln = self.ln1(x)
        
        # Self-attention with RoPE and explicit causal mask
        attn_output = self.attn(
            x=x_ln,
            freqs_cis=freqs_cis,
            mask=causal_mask
        )
        x = x + attn_output
        
        # Feed-forward block with pre-RMSNorm
        x = x + self.ffwd(self.ln2(x))
        
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Token embeddings
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        
        # No position embeddings needed since we use RoPE
        
        # Precompute complex rotations for RoPE
        self.freqs_cis = precompute_freqs_cis(n_embd // n_head, block_size)
        
        # Transformer blocks
        self.blocks = nn.ModuleList([
            Block(n_embd, n_head, gqa_kv_head) for _ in range(n_layer)
        ])
        
        # Final RMSNorm and output projection
        self.ln_f = RMSNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size, bias=False)
        
        # Generate causal mask once
        mask = torch.triu(torch.ones(block_size, block_size) * float('-inf'), diagonal=1)
        self.register_buffer("causal_mask", mask)
        
        # Initialize weights
        self.apply(self._init_weights)

        # Weight tying between embedding and output layer
        self.token_embedding_table.weight = self.lm_head.weight

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        device = idx.device
        
        # Obtain token embeddings (no position embeddings needed with RoPE)
        x = self.token_embedding_table(idx)  # (B, T, C)
        
        # Move freqs_cis to the correct device
        freqs_cis = self.freqs_cis.to(device)
        
        # Crop mask to current sequence length
        causal_mask = self.causal_mask[:T, :T]
        
        # Pass through transformer blocks
        for block in self.blocks:
            x = block(x, freqs_cis, causal_mask)
            
        # Final layer normalization and output projection to logits
        x = self.ln_f(x)  # (B, T, C)
        logits = self.lm_head(x)  # (B, T, vocab_size)

        # Compute loss if targets are provided
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    def generate(self, idx, max_new_tokens):
        """
        Given a sequence of indices 'idx', generate 'max_new_tokens' new tokens.
        """
        self.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            for _ in range(max_new_tokens):
                # Crop the sequence to the last block_size tokens
                idx_cond = idx[:, -block_size:]
                # Get predictions
                logits, _ = self(idx_cond)
                # Focus only on the last time step
                logits = logits[:, -1, :]  # (B, vocab_size)
                # Apply softmax to convert logits to probabilities
                probs = F.softmax(logits, dim=-1)  # (B, vocab_size)
                # Sample from the probability distribution
                idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
                # Append the new token to the sequence
                idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [23]:
torch.cuda.empty_cache()

In [24]:
model = GPTLanguageModel()

# if torch.cuda.device_count() > 1:
#     model = torch.nn.DataParallel(model)

model = model.to(device)
model = torch.compile(model)

print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

48.853504 M parameters


In [25]:
fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
use_fused = fused_available and 'cuda' == str(device)
print(f"{use_fused=}")

use_fused=True


In [26]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay, betas=(beta1, beta2), eps=1e-8, fused=use_fused)

In [27]:
true_total_steps = len(train_dataloader) // gradient_accumulation_steps
scheduler = lr_scheduler.OneCycleLR(
    optimizer, max_lr=6e-4, total_steps=true_total_steps, pct_start=0.05
)


In [28]:
os.makedirs("ckpt/", exist_ok=True)

In [29]:
str(device)

'cuda'

In [30]:
sample = tokenizer.decode(tokenizer.encode(ds["train"][0]["text"][:100], bos=True, eos=True))
sample

'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with<|endoftext|>'

In [31]:
def generate(model, idx, max_new_tokens):
    for _ in range(max_new_tokens):
        # crop idx to the last block_size tokens
        idx_cond = idx[:, -block_size:]
        # get the predictions
        logits, loss = model(idx_cond)
        # focus only on the last time step
        logits = logits[:, -1, :]  # becomes (B, C)
        # apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1)  # (B, C)
        # sample from the distribution
        idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
        # append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
    return idx

In [32]:
gradient_accumulation_steps, batch_size, target_batch_size

(256, 16, 4096)

In [33]:
with open("losses.txt", "w") as f:
	f.write("Step,Learing Rate,Training Loss,Validation Loss,Output\n")

In [34]:
for iter, batch in enumerate(tqdm.notebook.tqdm(train_dataloader, total=len(train_dataloader))):
    # inputs, targets = batch['text'], batch['text']
    inputs = batch['input'][:, :-1]
    targets = batch['input'][:, 1:]
    inputs, targets = inputs.to(device), targets.to(device)

    with torch.autocast(device_type=str(device), dtype=torch.bfloat16):
        logits, _ = model(inputs)
        loss = F.cross_entropy(
			logits.view(-1, logits.size(-1)),
			targets.view(-1),
			ignore_index=tokenizer.pad_id
		)

    loss = loss / gradient_accumulation_steps
    loss.backward()

    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    if (iter + 1) % gradient_accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()

    if iter % (gradient_accumulation_steps * 2) == 0 or iter == max_iters - 1:
        print(f"\nStep {iter}: Performing validation")
        print(f"Learning rate: {scheduler.get_last_lr()[0]:.6f}")
        model.eval()
        with torch.no_grad():
            val_loss = 0
            train_loss = loss.item() * gradient_accumulation_steps
            for batch in tqdm.notebook.tqdm(valid_dataloader, total=len(valid_dataloader)):
                # inputs, targets = batch['text'], batch['text']
                inputs = batch['input'][:, :-1]
                targets = batch['input'][:, 1:]
                inputs, targets = inputs.to(device), targets.to(device)
                logits, _ = model(inputs)
                loss = F.cross_entropy(
                    logits.view(-1, logits.size(-1)),
					targets.view(-1),
					ignore_index=tokenizer.pad_id
				)
                val_loss += loss.item()

            torch.save(model.state_dict(), f"ckpt/ckpt_{iter}.pt")
            print(f"Train loss: {train_loss:.4f}")
            print(f"Validation loss: {val_loss / len(valid_dataloader):.4f}")

            prompt = "One day, a dog"
            prompt = torch.tensor([encode(prompt)], dtype=torch.long, device=device)
            output = decode(generate(model, prompt, max_new_tokens=50)[0].tolist())
            print(output)
            output = output.replace("\n", "\n")
            output = output.replace('"', "'") 
            with open("losses.txt", "a") as f:
                f.write(f"{iter},{scheduler.get_last_lr()[0]:.6f},{train_loss},{val_loss / len(valid_dataloader)},\"{output}\"\n")
        model.train()

  0%|          | 0/132483 [00:00<?, ?it/s]


Step 0: Performing validation
Learning rate: 0.000024


  0%|          | 0/16 [00:00<?, ?it/s]

Train loss: 10.8579
Validation loss: 10.8531
One day, a dog roughly Bounty framing reliancedog See Within acet gunmen doneodorano shelfMerc Enchant Leonardoailloader Chall� failure Frank Jorge 286417 Elementary Secretary Aut visibility Flan afar Hubbardparable MetallExcellent /**justiceRICinavimedia beverage Eighth tort682 filters Solid designing Upton glances blindly

Step 512: Performing validation
Learning rate: 0.000033


  0%|          | 0/16 [00:00<?, ?it/s]

...

Train loss: 1.1407
Validation loss: 1.0231
One day, a dog named Tim woke up to a walk in the garden. He wanted to take a ride on the grass. He saw a big, brown swim in the neighborhood. He decided to climb it too. 

 arrows started to swim in the fence.

Step 131584: Performing validation
Learning rate: 0.000000


  0%|          | 0/16 [00:00<?, ?it/s]

Train loss: 1.1538
Validation loss: 1.0231
One day, a dog named Timmy went for a walk with his dad. They saw a big, ducks. Billy wanted to see! He said, "Let's wear his dad wraps me. But you come inside the park later." Timmy didn't want to go

Step 132096: Performing validation
Learning rate: 0.000000


  0%|          | 0/16 [00:00<?, ?it/s]

Train loss: 1.3599
Validation loss: 1.0231
One day, a dog named Max had a big coat driving on the truck. The dog felt sniffed the puddle and he put the wagon away in his outfits. The clinic, a delicious puddle with flour, a big smile. Max wanted to sleep on the p


In [35]:
torch.save(model.state_dict(), "final_model_tiny_stories_tiktoken_21052025_2.pt")

In [36]:
model = model.eval()

In [37]:
prompt = "There was a girl who"

prompt = torch.tensor([encode(prompt)], dtype=torch.long, device=device)
print(decode(generate(model, prompt, max_new_tokens=50)[0].tolist()))

/home/shrirang/Desktop/Projects/GPT Train/venv/lib/python3.12/site-packages/torch/_inductor/lowering.py:1814: UserWarning: Torchinductor does not support code generation for complex operators. Performance may be worse than eager.
  warnings.warn(
W0521 21:57:58.090000 1216536 torch/_inductor/utils.py:1137] [0/4] Not enough SMs to use max_autotune_gemm mode


There was a girl who loved to share her toys and keep her vegetables. One day she decided to write her shoes illness. She put a tiny staff in a sticker that was a turn!

The town was very curious and started to ride her car until day, a


In [38]:
prompt = "One day, a little girl named Lily found"

prompt = torch.tensor([encode(prompt)], dtype=torch.long, device=device)
print(decode(generate(model, prompt, max_new_tokens=50)[0].tolist()))

One day, a little girl named Lily found her favorite toy toy toy toy car. She was very happy and wanted to join it.

As Lily decided to go on the park, Lily tried to build a jacket. The car started to play next. She liked the toys and laughed happily
